In [211]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()  

login_url = 'https://login.pwr.edu.pl/auth/realms/pwr.edu.pl/protocol/cas/login?service=https%3A%2F%2Fweb.usos.pwr.edu.pl%2Fkontroler.php%3F_action%3Dlogowaniecas%2Findex&locale=pl'
driver.get(login_url)

time.sleep(3) 

In [212]:
username_field = driver.find_element(By.NAME, 'username')
password_field = driver.find_element(By.NAME, 'password')

In [213]:
username_field.send_keys('') #your login to usos here
password_field.send_keys('') #your password to usos here


In [214]:
driver.find_element(By.NAME, 'login').click()

In [215]:
#go to the class you want to scrape
driver.get('https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/przedmioty/pokazZajecia&zaj_cyk_id=72691&gr_nr=1')

In [216]:
from sqlalchemy import create_engine, text

# Database connection details
USER = "appuser"
PASSWORD = "apppassword"
DATABASE_NAME = "AttendMate"
DATABASE_URI = "mysql+pymysql://{}:{}@localhost:3306/{}".format(USER, PASSWORD, DATABASE_NAME)

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URI)


In [217]:
#get class atributes
table = driver.find_element(By.CLASS_NAME, "grey")  # Adjust the XPath if needed

# Get all rows inside the table
rows = table.find_elements(By.TAG_NAME, "tr")

table_data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")  # Or use "th" for headers
    table_data.append([cell.text for cell in cells])

for row in table_data:
    print(row)

['Przedmiot:', 'Software Engineering [W04IST-SI4023P]']
['Zajęcia:', 'Semestr zimowy 2024/25 [2024/25-Z] (w trakcie)\nProjekt [P], grupa nr 1 [pozostałe grupy]']
['Termin i miejsce:\ninfo', 'każdy poniedziałek, 17:05 - 18:45\nsala 127a\nBudynek dydaktyczny [D-2] jaki jest adres?']
['Terminy najbliższych spotkań:\ninfo', '2025-02-04 17:05 : 18:45 sala 127a\nBudynek dydaktyczny [D-2]\nCzęść spotkań jest ukryta - pokaż terminy wszystkich spotkań.', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '2025-02-04 17:05 : 18:45 sala 127a\nBudynek dydaktyczny [D-2]', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['2025-02-04 17:05 : 18:45 sala 127a\nBudynek dydaktyczny [D-2]', '']
['Liczba osób w grupie:', '15']
['Limit miejsc:', '14']
['Zaliczenie:', 'Zaliczenie']
['Prowadzący:', 'Marek Krótkiewicz']
['Moodle:', '(brak danych)']


In [218]:
subject = table_data[0][1].split()
subject_name = subject[0]+" "+subject[1]
class_code = subject[2][1:-1]
class_type = table_data[1][1].split('\n')[1].split(',')[0]

def class_type_to_enum(class_type : str):
    if class_type.lower().startswith("ćwiczenia"):
        return "lab"
    if class_type.lower().startswith("wykład"):
        return "lecture"
    if class_type.lower().startswith("seminarium"):
        return "seminar"
    if class_type.lower().startswith("projekt"):
        return "project"

class_type = class_type_to_enum(class_type)

print(class_type)

day = table_data[2][1].split()[1][0:-1]

def polish_to_english_day(day):
    if day == "poniedziałek":
        return "monday"
    if day == "wtorek":
        return "tuesday"
    if day == "środa":
        return "wednesday"
    if day == "czwartek":
        return "thursday"
    if day == "piątek":
        return "friday"
    if day == "sobota":
        return "saturday"
    if day == "niedziela":
        return "sunday"

day = polish_to_english_day(day)
class_start_time = table_data[2][1].split()[2]
room = table_data[2][1].split()[6]

def add_class_to_database(subject_name, class_type, class_code, day, class_start_time, room):
    with engine.begin() as connection:  # Ensures transactions are handled properly
        query = text("SELECT * FROM class WHERE subjectNumber = :class_code")
        result = connection.execute(query, {"class_code": class_code})
        existing_class = result.fetchone()  # Correct way to check if the class exists

        if existing_class is None:
            insert_query = text("""INSERT INTO class (subjectName, subjectType, subjectNumber, year, semester, room, day, time, absenceLimit) 
                                   VALUES (:subject_name, :class_type, :class_code, '2024', '2', :room, :day, :time, 3)""")
            connection.execute(insert_query, {
                "subject_name": subject_name,
                "class_type": class_type,
                "class_code": class_code,
                "room": room,
                "day": day,
                "time": class_start_time
            })

project


In [164]:
def add_student_to_database(first_name, last_name, email):
    with engine.connect() as connection:
        query = text("SELECT * FROM user WHERE email = :email")
        result = connection.execute(query, {"email": email})
        if result.rowcount == 0:
           with engine.begin() as connection:  # Automatically commits at the end
                query = text("INSERT INTO user (name, lastName, email, password) VALUES (:first_name, :last_name, :email, 'NULL')")
                connection.execute(query, {"first_name": first_name, "last_name": last_name, "email": email})

                query = text("INSERT INTO student (userID, studentNumber) values (LAST_INSERT_ID(), :student_number)")
                connection.execute(query, {"student_number": email[:6]})

def add_student_to_course(classNumber, email):
    with engine.connect() as connection:
        query = text("SELECT * FROM user WHERE email = :email")
        result = connection.execute(query, {"email": email})
        if result.rowcount == 1:
            user_id = result.fetchone()[0]
            query = text("SELECT * FROM student WHERE userID = :user_id")
            result = connection.execute(query, {"user_id": user_id})
            if result.rowcount == 1:
                student_id = result.fetchone()[0]
                query = text("SELECT classID FROM class WHERE subjectNumber = :classNumber")
                result = connection.execute(query, {"classNumber": classNumber})
                if result.rowcount == 1:
                    course_id = result.fetchone()[0]
                    with engine.begin() as connection:
                        query = text("INSERT INTO studentsInClasses (classID, studentID) VALUES (:class_id, :student_id)")
                        connection.execute(query, {"student_id": student_id, "class_id": course_id})

In [153]:
add_class_to_database(subject_name, class_type, class_code, day, class_start_time, room)

In [154]:
elements = driver.find_elements(By.CLASS_NAME, "wrgrey")
elements = [el.get_attribute("href") for el in elements if "osoby" in el.get_attribute("href")]

for el in elements:
    print(el)

https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=587778
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=563113
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=587785
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=563126
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=563131
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=612638
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=566991
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=612634
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=612609
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=612623
https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=612628
https://web.usos.pwr.

In [155]:
def parse_student(link):
    driver.get(link)
    time.sleep(1)
    names = driver.find_elements(By.CLASS_NAME, "uwb-clearfix")
    first_name = names[0].text.split()[1]
    last_name = names[1].text.split()[1]
    email = names[2].text
    add_student_to_database(first_name, last_name, email)
    add_student_to_course(class_code, email)
    driver.back()

In [165]:
for i in range(0, len(elements)):
    #it might crash when it loads your profile, so you can put link to your profile here 
    if(elements[i] and elements[i]!= ''):
        parse_student(elements[i])

In [191]:
driver.find_element(By.CLASS_NAME, "pokaz_wszystkie_daty").click()

In [192]:
table = driver.find_element(By.CLASS_NAME, "grey")  # Adjust the XPath if needed

# Get all rows inside the table

rows = table.find_elements(By.TAG_NAME, "tr")

table_data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")  # Or use "th" for headers
    table_data.append([cell.text for cell in cells])

In [194]:
import re

for row in table_data:
    d = re.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2} : \d{2}:\d{2}',row[0])
    if(d):
        date = d.group(0).split()
        print(date)
        with engine.connect() as connection:
            query = text("SELECT classID FROM class WHERE subjectNumber = :classNumber")
            result = connection.execute(query, {"classNumber": class_code})
            if result.rowcount == 1:
                course_id = result.fetchone()[0]
                with engine.begin() as connection:
                    query = text("INSERT INTO ClassSession (classID, sessionDate, sessionStartTime, sessionEndTime) VALUES (:class_id, :date, :start_time, :end_time)")
                    connection.execute(query, {"class_id": course_id, "date": date[0], "start_time": date[1], "end_time": date[3]})

['2024-10-07', '17:05', ':', '18:45']
['2024-10-14', '17:05', ':', '18:45']
['2024-10-21', '17:05', ':', '18:45']
['2024-10-28', '17:05', ':', '18:45']
['2024-11-04', '17:05', ':', '18:45']
['2024-11-18', '17:05', ':', '18:45']
['2024-11-25', '17:05', ':', '18:45']
['2024-12-02', '17:05', ':', '18:45']
['2024-12-09', '17:05', ':', '18:45']
['2024-12-16', '17:05', ':', '18:45']
['2025-01-13', '17:05', ':', '18:45']
['2025-01-20', '17:05', ':', '18:45']
['2025-01-27', '17:05', ':', '18:45']
['2025-01-31', '17:05', ':', '18:45']
['2025-02-04', '17:05', ':', '18:45']


In [219]:
def add_teacher_to_database(first_name, last_name, email):
    with engine.connect() as connection:
        query = text("SELECT * FROM user WHERE email = :email")
        result = connection.execute(query, {"email": "email"})
        if result.rowcount == 0:
            with engine.begin() as connection:
                query = text("INSERT INTO user (name, lastName, email, password) VALUES (:first_name, :last_name, :email, 'NULL')")
                connection.execute(query, {"first_name": first_name, "last_name": last_name, "email": email})

                query = text("INSERT INTO teacher (userID) values (LAST_INSERT_ID())")
                connection.execute(query)

def add_teacher_to_course(classNumber, email):
    with engine.connect() as connection:
        query = text("SELECT * FROM user WHERE email = :email")
        result = connection.execute(query, {"email": email})
        if result.rowcount == 1:
            user_id = result.fetchone()[0]
            query = text("SELECT * FROM teacher WHERE userID = :user_id")
            result = connection.execute(query, {"user_id": user_id})
            if result.rowcount == 1:
                teacher_id = result.fetchone()[0]
                query = text("SELECT classID FROM class WHERE subjectNumber = :classNumber")
                result = connection.execute(query, {"classNumber": classNumber})
                if result.rowcount == 1:
                    course_id = result.fetchone()[0]
                    with engine.begin() as connection:
                        query = text("INSERT INTO teachersInClasses (classID, teacherID) VALUES (:class_id, :teacher_id)")
                        connection.execute(query, {"teacher_id": teacher_id, "class_id": course_id})
    

In [220]:
tr_element = driver.find_element(By.XPATH, "//tr[td[contains(text(), 'Prowadzący:')]]")

link_element = tr_element.find_element(By.XPATH, ".//td[2]/a")

href_value = link_element.get_attribute("href")
print("Lecturer's profile link:", href_value)

def parse_lecturer(link):
    driver.get(link)
    time.sleep(1)
    names = driver.find_elements(By.CLASS_NAME, "uwb-clearfix")
    first_name = names[0].text.split()[1]
    last_name = names[1].text.split()[1]
    email = names[3].text
    add_teacher_to_database(first_name, last_name, email)
    add_teacher_to_course(class_code, email)

parse_lecturer(href_value)

Lecturer's profile link: https://web.usos.pwr.edu.pl/kontroler.php?_action=katalog2/osoby/pokazOsobe&os_id=1644
